In [ ]:
import pandas as pd
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

In [ ]:
parquet_file = "query_result.parquet"
df = pd.read_parquet(parquet_file)

In [ ]:
csv_file = 'output_file.csv'
df.to_csv(csv_file, index=False)

In [ ]:
corpus = open("output_file.csv", "r").readlines()[1:]

In [23]:
import bm25s
import torch
import pandas as pd
from transformers import BertTokenizer

In [3]:
bm25 = bm25s.BM25.load("knowledge_bm25", load_corpus=True)

In [8]:
df = pd.read_csv("train.csv")
question_1 = df['question1']
question_2 = df['question2']
values = torch.tensor(df['is_duplicate']).double()

In [14]:
question_pair = [str(q1) + " [SEP] " + str(q2) for q1, q2 in zip(question_1, question_2)]

In [17]:
bm25_index, bm25_score = bm25.retrieve(bm25s.tokenize(question_pair), k = 5)

Split strings:   0%|          | 0/404290 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/404290 [00:00<?, ?it/s]

In [19]:
import pickle
with open('bm25_index.pickle', 'wb') as handle:
    pickle.dump(bm25_index, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('bm25_score.pickle', 'wb') as handle:
    pickle.dump(bm25_score, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
from tqdm import tqdm

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bm25_input_ids = [[] for _ in range(len(bm25_index))]
bm25_token_type_ids = [[] for _ in range(len(bm25_index))]
bm25_attention_mask = [[] for _ in range(len(bm25_index))]
for i in tqdm(range(len(bm25_index))):
    if (type(bm25_attention_mask[i]) == torch.Tensor):
        assert(bm25_input_ids[i].shape == (5, 128))
        assert(bm25_token_type_ids[i].shape == (5, 128))
        assert(bm25_attention_mask[i].shape == (5, 128))
        continue
    for j in range(len(bm25_index[i])):
        potato = tokenizer(bm25_index[i, j]['text'], max_length=128, truncation=True, padding="max_length", return_tensors="pt")
        bm25_input_ids[i].append(torch.tensor(potato['input_ids']))
        bm25_token_type_ids[i].append(torch.tensor(potato['token_type_ids']))
        bm25_attention_mask[i].append(torch.tensor(potato['attention_mask']))
    bm25_input_ids[i] = torch.stack(bm25_input_ids[i], dim=0).squeeze(1)
    bm25_token_type_ids[i] = torch.stack(bm25_token_type_ids[i], dim=0).squeeze(1)
    bm25_attention_mask[i] = torch.stack(bm25_attention_mask[i], dim=0).squeeze(1)


  0%|          | 0/404290 [00:00<?, ?it/s]/tmp/ipykernel_4688/3652585381.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bm25_input_ids[i].append(torch.tensor(potato['input_ids']))
/tmp/ipykernel_4688/3652585381.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bm25_token_type_ids[i].append(torch.tensor(potato['token_type_ids']))
/tmp/ipykernel_4688/3652585381.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bm25_attention_mask[i].append(torch.tensor(potato['attention_mask']))
 76%|███████▌  | 308156/404290 [22:13<06:55, 231.1

KeyboardInterrupt: 

In [37]:
bm25 = None
print(bm25_input_ids[0].shape)
print(bm25_token_type_ids[0].shape)
print(bm25_attention_mask[0].shape)
print(type(bm25_input_ids[0]) == torch.Tensor)

torch.Size([5, 128])
torch.Size([5, 128])
torch.Size([5, 128])
True


In [ ]:
torch.save({
    'bm25_score': torch.from_numpy(bm25_score),
    'bm25_input_ids':
})

<class 'numpy.ndarray'>
